<a href="https://colab.research.google.com/github/dhyeyvshah/comp560-final-project/blob/main/Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # necessary when matplotlib is imported
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set plot aesthetics
plt.rcParams['figure.dpi'] = 100

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samithsachidanandan/netflix-stock-price-2002-2025")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv('Netflix Inc. (NFLX) Stock Price 2002-2025.csv', encoding='utf-8-sig')

# Clean column names by stripping any extra whitespace
df.columns = [col.strip() for col in df.columns]

# Convert the Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Convert numeric columns from string to float (this will handle commas if present)
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
for col in numeric_columns:
    # Remove any commas and extra spaces, then convert to numeric
    df[col] = pd.to_numeric(df[col].str.replace(',', '').str.strip(), errors='coerce')

# Quick check of the DataFrame structure
print('DataFrame loaded with shape:', df.shape)
print('Columns:', df.columns.tolist())

In [ ]:
print('Missing values per column:')
print(df.isnull().sum())

# Drop rows with missing Date or Close values (as these are critical)
df = df.dropna(subset=['Date', 'Close']).reset_index(drop=True)

# Sort data by Date
df = df.sort_values('Date')

# Create a new feature: number of days since the first observation
df['Days'] = (df['Date'] - df['Date'].min()).dt.days

print('\nDataFrame shape after cleaning:', df.shape)

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df['Close'], kde=True, color='blue')
plt.title('Distribution of Close Prices')
plt.xlabel('Close Price')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('hist_close.png')
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(df['Volume'], kde=True, color='orange')
plt.title('Distribution of Volume')
plt.xlabel('Volume')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('hist_volume.png')
plt.show()

# Box Plot to examine outliers in 'Close' prices
plt.figure(figsize=(8, 4))
sns.boxplot(x=df['Close'], color='green')
plt.title('Box Plot of Close Prices')
plt.tight_layout()
plt.savefig('boxplot_close.png')
plt.show()

# Create a correlation heatmap for numeric columns if there are four or more
numeric_df = df.select_dtypes(include=[np.number])
if numeric_df.shape[1] >= 4:
    plt.figure(figsize=(10, 8))
    sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Heatmap for Numeric Features')
    plt.tight_layout()
    plt.savefig('correlation_heatmap.png')
    plt.show()

# Pair Plot for key features
sns.pairplot(df[['Days', 'Open', 'Close', 'Adj Close']])
plt.savefig('pairplot.png')
plt.show()

In [ ]:
df['Daily_Return'] = (df['Close'] - df['Open']) / df['Open']
df['5_MA'] = df['Close'].rolling(window=5).mean()
df['10_MA'] = df['Close'].rolling(window=10).mean()
df['Volatility'] = df['Close'].rolling(window=5).std()

# Drop NaN values caused by rolling window calculations
df.dropna(inplace=True)

# Create Target Variable (1 for Up, 0 for Down)
df['Price_Movement'] = (df['Close'].shift(-1) > df['Close']).astype(int)

# Select Features and Target
features = ['Open','Close', 'Daily_Return', '5_MA', '10_MA', 'Volatility']
X = df[features]
y = df['Price_Movement']

# Split into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10, shuffle=True)

# Train Naïve Bayes Model
model = GaussianNB()
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
features = ['Daily_Return', '5_MA', '10_MA']
X = df[features].values
y = df['Price_Movement'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Define the Gaussian Naive Bayes Classifier with Mini-Batch Gradient Descent
class MiniBatchGaussianNB:
    def __init__(self, batch_size=32, learning_rate=0.01, epochs=100):
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        self.means = {c: np.zeros(n_features) for c in self.classes}
        self.variances = {c: np.ones(n_features) for c in self.classes}
        self.priors = {c: np.sum(y == c) / n_samples for c in self.classes}

        for epoch in range(self.epochs):
            indices = np.random.permutation(n_samples)
            X_shuffled, y_shuffled = X[indices], y[indices]

            for i in range(0, n_samples, self.batch_size):
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]

                for c in self.classes:
                    X_c = X_batch[y_batch == c]
                    if len(X_c) == 0:
                        continue

                    # Compute gradients for mean and variance
                    grad_mean = np.mean(X_c, axis=0) - self.means[c]
                    grad_var = np.var(X_c, axis=0) - self.variances[c]

                    # Gradient descent updates
                    self.means[c] += self.learning_rate * grad_mean
                    self.variances[c] += self.learning_rate * grad_var

            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Mean update {self.means}, Variance update {self.variances}")

    def predict(self, X):
        probs = np.zeros((X.shape[0], len(self.classes)))

        for i, c in enumerate(self.classes):
            mean, var = self.means[c], self.variances[c]
            likelihood = (1 / np.sqrt(2 * np.pi * var)) * np.exp(-((X - mean) ** 2) / (2 * var))
            probs[:, i] = np.prod(likelihood, axis=1) * self.priors[c]

        return np.argmax(probs, axis=1)

# Train the Model
model = MiniBatchGaussianNB(batch_size=32, learning_rate=0.0001, epochs=100)
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))